In [1]:
import os
import getpass

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

from langchain_chroma import Chroma

from datetime import datetime, timedelta
import requests

In [2]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GEMINI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')
WEATHER_API = os.getenv('WEATHER_API')
BASE_URL = "http://api.weatherapi.com/v1/forecast.json"

In [3]:
def get_date_for_day(day_name):
    """
    Função para construir a data para um dia da semana específico.
    """
    today = datetime.today()
    today_weekday = today.weekday()

    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    try:
        requested_weekday = days_of_week.index(day_name)
    except ValueError:
        return None

    if requested_weekday == today_weekday:
       return today.strftime("%Y-%m-%d")
    
    days_until_requested = (requested_weekday - today_weekday + 7) % 7
    target_date = today + timedelta(days=days_until_requested)
    print(target_date)
    return target_date.strftime("%Y-%m-%d")

In [4]:
def weatherapi_forecast(date_string: str) -> str:
    """
    Ferramenta customizada que usa a WeatherAPI para obter a previsão do tempo
    para a cidade de Natal em uma data específica usando a biblioteca requests.

    Args:
        date_string (str): Uma string contendo a data no formato yyyy-mm-dd.

    Returns:
        str: Uma string contendo a previsão do tempo para a data especificada.
    """

    try:
        params = {
            "key": WEATHER_API,
            "q": 'Natal',
            "dt": date_string,
            "aqi": "no",
            "alerts": "no"
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Lança um erro para códigos de status HTTP ruins (4xx ou 5xx)
        data = response.json()

        if data and data.get("forecast") and data["forecast"].get("forecastday"):
             forecast_data = data["forecast"]["forecastday"][0]
             return (
                 f"Previsão para {date_string}:\n"
                 f"  Temperatura média: {forecast_data['day']['avgtemp_c']}°C\n"
                 f"  Condição: {forecast_data['day']['condition']['text']}\n"
                 f"  Chance de chuva: {forecast_data['day']['daily_chance_of_rain']}%\n"
                 f"  Umidade média: {forecast_data['day']['avghumidity']}%\n"
                 f"  Velocidade média do vento: {forecast_data['day']['maxwind_kph']} km/h"
            )
        else:
            return "Previsão não encontrada para essa data."
    except requests.exceptions.RequestException as e:
        return f"Erro ao buscar informações meteorológicas: {str(e)}"
    except Exception as e:
        return f"Erro inesperado: {str(e)}"

In [5]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

# Caso queira utilizar uma LLM local com o Ollama
# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model="llama3.2", temperature=0.6, max_tokens=200)

In [6]:
from get_embedding_function import get_embedding_function
CHROMA_PATH = "chroma"

def query_rag(query_text):
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    return context_text

def query_rag_tool(query_text: str) -> str:
    return query_rag(query_text)

In [7]:
ddg_search = DuckDuckGoSearchAPIWrapper()

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Use esta ferramenta para buscar informações recentes e responda ao usuário com os resultados encontrados. É importante que a resposta contenha o que foi recuperado da pesquisa."
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast,
        description="Você deve utilizar essa ferramenta quando o usuário solicitar informações sobre o clima. Se o usuário pedir informações para um período de tempo (por exemplo, 'de sexta a domingo'), você DEVE usar a ferramenta separadamente para cada dia desse período e depois combinar os resultados na resposta final. Se o usuário fornecer apenas uma data, use a ferramenta apenas uma vez."
    ),
    Tool(
        name="Query RAG",
        func=query_rag_tool,
        description="A perguntas relacionadas ao turismo da cidade de Natal no Rio Grande do Norte. Se as informações não forem suficientes no Query RAG para responder ao usuário utilize a ferramenta do DuckDuckGo Search"
    )
]

In [8]:
agent_prompt = hub.pull("tales/agente_turismo")

In [9]:
print(agent_prompt)

input_variables=['agent_scratchpad', 'chat_history', 'data_atual', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'tales', 'lc_hub_repo': 'agente_turismo', 'lc_hub_commit_hash': '01a133cdf82e821cdb37a291a75b2f86c7d0327b475cc84015bb3ca21cba5167'} template='Seu nome é Sofia. Você é uma assistente pessoal de inteligência artificial de classe mundial, especializada em viagens e turismo. Você é altamente avançada e inteligente, muito útil e eficiente em fornecer informações, planejar roteiros personalizados e resolver problemas relacionados a viagens. Sofia é um modelo AELM (Modelo de Linguagem de Execução Automática) com foco em planejamento de viagens.\n\n"Sofia" foi projetada para auxiliar usuários em todas as etapas de suas aventuras, desde a escolha do destino até a criação de itinerários detalhados e dicas práticas. Como modelo de linguagem, "Sofia" é capaz de gerar texto semelhante ao humano com base nas informações fornecidas pelos usuá

In [10]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [11]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=5, chat_memory=history, memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_8944\582812787.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=5, chat_memory=history, memory_key="chat_history")


In [12]:
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_atual = f"{dia_da_semana_pt}, {data_formatada}"

In [13]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [14]:
print(data_atual)

sábado, 2024/12/28


In [15]:
response = agent_executor.invoke({"input": f"""
                                  Boa noite
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Boa noite!  Qual cidade você pretende visitar?


> Finished chain.
Boa noite!  Qual cidade você pretende visitar?


In [16]:
response = agent_executor.invoke({"input": f"""
                                  Eu estou planejando visitar Natal no Rio Grande do Norte
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Ótimo! Natal é uma cidade linda. Quantos dias você planeja passar lá, e quais os dias da semana?


> Finished chain.
Ótimo! Natal é uma cidade linda. Quantos dias você planeja passar lá, e quais os dias da semana?


In [17]:
response = agent_executor.invoke({"input": f"""
                                  Eu planejo passar da segunda dia 30 até a quarta
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Perfeito! De segunda-feira (30/12) até quarta-feira (01/01).  Quais são seus interesses principais em Natal? Por exemplo, praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```


> Finished chain.
Perfeito! De segunda-feira (30/12) até quarta-feira (01/01).  Quais são seus interesses principais em Natal? Por exemplo, praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```


In [18]:
response = agent_executor.invoke({"input": f"""
                                  Eu planejo vizitar praias e restaurantes regionais
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: Quais são os melhores restaurantes regionais em Natal e as melhores praias para visitar?PRAIA DA REDINHA
A última praia ao norte de Natal é uma antiga vila de pescadores, cujo destaque é o 
Mercado Público. A tradição ali é comer a ginga com tapioca (peixe frito que vem no 
palito, dentro de uma tapioca). A Igreja de Nossa Senhora dos Navegantes, construída 
com pedras dos arrecifes, e o Aquário Natal são outros pontos de interesse.
PRAIA DO MEIO
Situada entre a Praia do Forte 
e a Praia dos Artistas, sedia 
anualmente, durante o Réveillon, 
a tradicional festa de Iemanjá. Era 
considerada a praia mais turística 
da cidade até os anos 1980, 
devido às formações de piscinas 
naturais da região.
PRAIA DOS ARTISTAS
A 5 km do centro de Natal, é uma  
das mais agitadas da cidade. Suas 
ondas altas são ótimas para surfe 
e outros esportes náuticos. Com

---

PRAIA A PRAIA
A orla urbana de Natal é marcada por c

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: Qual o orçamento aproximado para essa viagem?FOTO: DUNAS DE GENIPABU / EMPROTUR/SETUR
GUIA COMPLETO
Os melhores passeios, restaurantes,
hotéis, dicas de roteiros e muito mais
NATAL

---

O QUE 
FAZEMOS 
POR 
VOCÊ
CVC
A maior agência de turismo da América Latina
C
V
C
FOTO: BARCO AO PÔR DO SOL / EMPROTUR/SETUR
ASSISTÊNCIA 24H
A CVC está com você antes, 
durante e após a viagem, sempre 
resolvendo tudo.
VIAGENS
PERSONALIZADAS
Montamos a viagem como você 
quiser. Você define o hotel, passeios, 
transporte individual ou em grupo, 
guia e todos os detalhes. 
HOTÉIS
SELECIONADOS
Escolhemos as melhores hospedagens 
de cada categoria. Todas as opções têm 
ótimo custo/benefício e são sempre 
monitoradas pela CVC.
PASSEIOS
Nossos especialistas descobrem os 
passeios mais incríveis dos destinos, e 
você curte tudo com muito conforto.
UMA VIAGEM
COMPLETA
Você encontra tudo o que deseja para 
sua viagem na CVC, como 

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Entendi! Praias e restaurantes regionais são ótimos interesses para uma viagem a Natal.  Para te ajudar a planejar melhor, preciso saber qual o seu orçamento aproximado para essa viagem?
```


> Finished chain.
Entendi! Praias e restaurantes regionais são ótimos interesses para uma viagem a Natal.  Para te ajudar a planejar melhor, preciso saber qual o seu orçamento aproximado para essa viagem?
```


In [19]:
response = agent_executor.invoke({"input": f"""
                                  Sem incluir Hóteis pretendo gastar no máximo 1000 reais
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: orçamento para viagem de 3 dias em Natal focado em praias e restaurantes regionais, sem incluir hospedagem.são para charretes, e não para carros. Inclui: transporte de ida e volta ao hotel, guia e 
passeio de barco. Duração: dia inteiro. 
RIO DO FOGO
Na vila de pescadores situada a  
85 km de Natal, você passa o dia  
na Praia de Perobas, onde 
mergulha em águas cristalinas, 
aproveita um restaurante pé na 
areia e pode fazer um passeio 
opcional de lancha aos parrachos. 
Inclui: transporte de ida e volta ao 
hotel e guia. Duração: dia inteiro.
PASSEIOS
BARRA DO PUNAÚ 
Durante o trajeto de 70 km até a Barra 
do Rio Punaú, você visita o Aquário 
Natal, que é o maior do Nordeste 
(ingresso pago no local). Na chegada a 
Punaú, pode aproveitar as águas mornas 
do rio e fazer um passeio opcional de 
helicóptero. Inclui: transporte de ida e 
volta ao hotel, guia e entrada na Fazenda

---

8 DIAS (7 NOITES) 
6º

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-30Previsão para 2024-12-30:
  Temperatura média: 27.8°C
  Condição: Patchy rain nearby
  Chance de chuva: 87%
  Umidade média: 70%
  Velocidade média do vento: 23.8 km/h

...

```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-31Previsão para 2024-12-31:
  Temperatura média: 28.1°C
  Condição: Patchy rain nearby
  Chance de chuva: 89%
  Umidade média: 67%
  Velocidade média do vento: 24.5 km/h

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2025-01-01Previsão para 2025-01-01:
  Temperatura média: 28.2°C
  Condição: Patchy rain nearby
  Chance de chuva: 86%
  Umidade média: 67%
  Velocidade média do vento: 22.0 km/h

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Com R$1000,00 para gastar em praias e restaurantes, podemos criar um roteiro incrível em Natal.  Considerando a previsão do tempo, com alta chance de chuva nos três dias (30/12, 31/12 e 01/01), precisamos planejar atividades que sejam menos dependentes do sol.

Para começar, vamos definir as atividades para cada dia.  Para isso, preciso saber: você prefere explorar diferentes praias todos os dias ou se concentrar em uma ou duas, aproveitando mais a fundo?
```


> Finished chain.
Ótimo! Com R$1000,00 para gastar em praias e restaurantes, podemos criar um roteiro incrível em Natal.  Considerando a previsão do tempo, com alta chance de chuva nos três dias (30/12, 31/12 e 01/01), precisamos planejar atividades que sejam menos dependentes do sol.

Para começar, vamos definir as atividades para cada dia.  Para isso, preciso saber: você prefere explorar diferentes praias todos os dias ou se concentrar em uma ou duas, aprov

In [20]:
response = agent_executor.invoke({"input": f"""
                                  Eu quero aproveitar uma praia por dia e relaxar
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-30Previsão para 2024-12-30:
  Temperatura média: 27.8°C
  Condição: Patchy rain nearby
  Chance de chuva: 87%
  Umidade média: 70%
  Velocidade média do vento: 23.8 km/h

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-31Previsão para 2024-12-31:
  Temperatura média: 28.1°C
  Condição: Patchy rain nearby
  Chance de chuva: 89%
  Umidade média: 67%
  Velocidade média do vento: 24.5 km/h

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2025-01-01Previsão para 2025-01-01:
  Temperatura média: 28.2°C
  Condição: Patchy rain nearby
  Chance de chuva: 86%
  Umidade média: 67%
  Velocidade média do vento: 22.0 km/h

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ok, com a previsão de chuva para os três dias, vamos focar em atividades que possam ser aproveitadas mesmo com chuva.  Como você quer aproveitar uma praia por dia e relaxar, vamos escolher praias com infraestrutura que permita isso, mesmo com chuva. Sugiro o seguinte roteiro:

**Segunda-feira (30/12):**

* **Manhã:** Chegada em Natal e check-in no hotel (considerando que você já tenha reservado).  Como a previsão do tempo indica chuva, sugiro ir direto para um restaurante regional para almoçar.
* **Tarde:** Praia de Ponta Negra.  Mesmo com chuva, você pode aproveitar o passeio de jangada, visitar os quiosques (alguns têm áreas cobertas) e apreciar a vista.  Caso a chuva esteja muito forte, você pode aproveitar para fazer compras em alguma loja na região.
* **Noite:** Jantar em um restaurante regional em Ponta Negra.  Há muitas opções com frutos do mar frescos e pratos típicos da região.  Sugiro pesquisar opções com base em

In [22]:
response = agent_executor.invoke({"input": f"""
                                  Obrigado
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: De nada!  Espero que você tenha uma ótima viagem para Natal.  Qualquer dúvida ou ajuste no roteiro, pode me perguntar!
```


> Finished chain.
De nada!  Espero que você tenha uma ótima viagem para Natal.  Qualquer dúvida ou ajuste no roteiro, pode me perguntar!
```


In [23]:
print(response["chat_history"])

Human: 
                                  Eu estou planejando visitar Natal no Rio Grande do Norte
                                  
AI: Ótimo! Natal é uma cidade linda. Quantos dias você planeja passar lá, e quais os dias da semana?
Human: 
                                  Eu planejo passar da segunda dia 30 até a quarta
                                  
AI: Perfeito! De segunda-feira (30/12) até quarta-feira (01/01).  Quais são seus interesses principais em Natal? Por exemplo, praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```
Human: 
                                  Eu planejo vizitar praias e restaurantes regionais
                                  
AI: Entendi! Praias e restaurantes regionais são ótimos interesses para uma viagem a Natal.  Para te ajudar a planejar melhor, preciso saber qual o seu orçamento aproximado para essa viagem?
```
Human: 
                                  Sem incluir Hóteis pretendo gastar no máximo 100